In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#import tensorflow_hub as hub
import pandas as pd
import numpy as np
import chess

In [2]:
def boardstate(fen):
    board = chess.Board(fen[0])
    fstr = str(fen[0])

    if board.has_kingside_castling_rights(chess.WHITE) == True:
        WCKI = 1
    else:
        WCKI = 0
    if board.has_queenside_castling_rights(chess.WHITE) == True:
        WCQ = 1
    else:
        WCQ = 0
    if board.is_check() == True:
        WCH = 1
    else:
        WCH = 0

    if board.has_kingside_castling_rights(chess.BLACK) == True:
        BCKI = 1
    else:
        BCKI = 0
    if board.has_queenside_castling_rights(chess.BLACK) == True:
        BCQ = 1
    else:
        BCQ = 0
    if board.was_into_check() == True:
        BCH = 1
    else:
        BCH = 0

    #f = [M, WCKI, WCQ, WCH, BCKI, BCQ, BCH]
    fw = [WCKI, WCQ, WCH]
    fb = [BCKI, BCQ, BCH]

    bstr = str(board)
    bstr = bstr.replace("p", "\ -1")
    bstr = bstr.replace("n", "\ -3")
    bstr = bstr.replace("b", "\ -4")
    bstr = bstr.replace("r", "\ -5")
    bstr = bstr.replace("q", "\ -9")
    bstr = bstr.replace("k", "\ -100")
    bstr = bstr.replace("P", "\ 1")
    bstr = bstr.replace("N", "\ 3")
    bstr = bstr.replace("B", "\ 4")
    bstr = bstr.replace("R", "\ 5")
    bstr = bstr.replace("Q", "\ 9")
    bstr = bstr.replace("K", "\ 100")
    bstr = bstr.replace(".", "\ 0")
    bstr = bstr.replace("\ ", ",")
    bstr = bstr.replace("'", " ")
    bstr = bstr.replace("\n", "")
    bstr = bstr.replace(" ", "")
    bstr = bstr[1:]
    bstr = eval(bstr)
    bstr = list(bstr)
    if "w" not in fstr:
        for i in range(len(bstr)):
            bstr[i] = bstr[i] * -1
        bstr.reverse()
        fs = fb
        fb = fw
        fw = fs


    BITBOARD = fw + fb + bstr


    return BITBOARD

In [3]:
#x = ''
#r = 1
def strfix(tr):
    if '#' in str(tr):
        if '-' in tr:
            t = -1000
        else:
            t = 1000
    elif '\ufeff+23' in str(tr):
        t = 0
    else:
        t = int(tr)
    
    t = t/100
    #try:
    #    t = int(t)
    #except ValueError:
    #    t = 0

    return t
#print(x)

In [15]:
data = pd.read_csv("data\chessData.csv")

# print(data)

label_columns = [1]

data_features = data.drop(columns=data.iloc[:, label_columns])

data_features = data_features.head(1000000)

data_labels = data.iloc[:, label_columns]

data_labels = data_labels.head(1000000)

data_labels = data_labels.astype(str) 


data_labels = data_labels.apply(strfix, axis=1)


#print(data_features)

#print(data_labels)

#print(data_labels.dtypes)

data_features = data_features.apply(boardstate, axis=1)

data_features = data_features.apply(pd.Series)

#print(data_features)

input2_columns = [0, 1, 2, 3, 4, 5]


inputboard = data_features.drop(columns=data_features.iloc[:, input2_columns]) 

#print(inputboard.dtypes)

inputboard = np.array(inputboard)
#print(inputboard.shape)

#print(inputboard)


inputmeta = data_features.iloc[:, input2_columns]

#print(inputmeta.dtypes)

inputmeta = np.array(inputmeta)
#print(inputmeta.shape)

print(inputmeta)

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
import numpy as np

input1 = tf.keras.layers.Input(shape=(64,))
shape1 = tf.keras.layers.Reshape(target_shape=(8, 8, 1))(input1)
conv1 = tf.keras.layers.Conv2D(kernel_size=(8,8), padding="same", activation="relu", filters=64, input_shape=(8,8,1))(shape1)
bn1 = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-05)(conv1)
conv2 = tf.keras.layers.Conv2D(kernel_size=(8,8), padding="same", activation="relu", filters=64, input_shape=(8,8,1))(bn1)
bn2 = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-05)(conv2)
flatten1 = tf.keras.layers.Flatten()(bn2)
input2 = tf.keras.layers.Input(shape=(6,))

conc = tf.keras.layers.concatenate([flatten1,input2])

Denselayer1 = tf.keras.layers.Dense(1024, activation='relu')(conc)
Denselayer2 = tf.keras.layers.Dense(512, activation='relu')(Denselayer1)
Denselayer3 = tf.keras.layers.Dense(256, activation='relu')(Denselayer2)
Denselayer4 = tf.keras.layers.Dense(256, activation='relu')(Denselayer3)
Output = tf.keras.layers.Dense(1, activation='linear')(Denselayer4)



data_model = tf.keras.models.Model(inputs=[input1, input2], outputs=Output)

predictions = data_model([(inputboard[:1]), (inputmeta[:1])]).numpy

metric =[tf.keras.metrics.MeanAbsoluteError()]

opt = tf.keras.optimizers.Adam(clipnorm=1)

los = tf.keras.losses.MeanSquaredError()

data_model.compile(optimizer=opt, 
                   loss=los,
                   metrics=metric)
data_model.summary()
data_model.fit([inputboard, inputmeta], data_labels, epochs=1000, batch_size=8192, shuffle=True)

data_model.save("engine01")